In [1]:
import numpy as np
from pyomo.environ import *
import pandas as pd

In [15]:
def load_distance_time_matrix(path, N):
    data= pd.read_csv(path)
    distance = np.zeros((N,N))
    time = np.zeros((N,N))
    
    for i in range(len(data)):
        origen = int(data.iloc[i, 0])
        destino = int(data.iloc[i, 1])
        distance[origen-1, destino-1] = float(data.iloc[i, 2])
        time[origen-1, destino-1] = float(data.iloc[i, 3])
    return distance, time

def load_vehicles(path):
    data= pd.read_csv(path)
    N = len(data)
    vehicles = {}
    for i in range(N):
        id = int(data.iloc[i, 0])
        capacity = int(data.iloc[i, 1])
        ran = float(data.iloc[i, 2])
        vehicles[id] = (capacity, ran)
    
    return vehicles

def load_demand(clientsPath):
    data= pd.read_csv(clientsPath)
    N = len(data)
    demand_dic = {}
    for i in range(N):
        id = int(data.iloc[i, 1])
        demand = float(data.iloc[i, 2])
        demand_dic[id] = demand
    
    return demand_dic


def load_capacity(depositPath):
    data= pd.read_csv(depositPath)
    N = len(data)
    capacity_dic = {}
    for i in range(N):
        id = int(data.iloc[i, 1])
        capacity = float(data.iloc[i, 4])
        capacity_dic[id] = capacity
    
    return capacity_dic

def load_distance_time_dic(path, N):
    data= pd.read_csv(path)
    distance = {}
    time = {}
    
    for i in range(N):
        origen = int(data.iloc[i, 0])
        destino = int(data.iloc[i, 1])
        distance[origen, destino] = float(data.iloc[i, 2])
        time[origen, destino] = float(data.iloc[i, 3])
    
    return distance, time

def load_coordinates(depotsPath, clientsPath):
    coord = {}
    depot= pd.read_csv(depotsPath)
    client= pd.read_csv(clientsPath)
    
    for i in range(len(depot)):
        id = int(depot.iloc[i, 1])
        lat = float(depot.iloc[i, 3])
        long = float(depot.iloc[i, 2])
        coord[id] = [lat, long]

    for j in range(len(client)):
        id = int(client.iloc[j, 1])
        lat = float(client.iloc[j, 4])
        long = float(client.iloc[j, 3])
        coord[id] = [lat, long]
    
    return coord
        

In [20]:
#Cargar datos
distancia,_ = load_distance_time_dic('../Datos/Caso2/caso2.csv', 21)
vehiculos = load_vehicles('../Datos/Caso2/vehicles.csv')
demanda = load_demand('..\Datos\Caso2\clients.csv')
oferta = load_capacity('..\Datos\Caso2\depots.csv')
coord = load_coordinates('..\Datos\Caso2\depots.csv', '..\Datos\Caso2\clients.csv')

print(f"Distancia: {distancia}")
print(f"Vehiculos: {vehiculos}")
print(f"Oferta: {oferta}")
print(f"Demanda: {demanda}")


Distancia: {(1, 2): 33061.2, (1, 3): 10160.7, (1, 4): 6452.9, (1, 5): 21415.6, (1, 6): 16924.1, (1, 7): 20126.9, (1, 8): 11361.2, (1, 9): 4329.9, (1, 10): 31289.5, (1, 11): 13659.0, (1, 12): 26223.2, (1, 13): 22414.9, (1, 14): 25897.0, (1, 15): 4225.5, (1, 16): 22338.8, (1, 17): 12064.8, (1, 18): 18088.6, (1, 19): 19570.0, (1, 20): 10485.2, (1, 21): 18553.2, (2, 1): 32946.7}
Vehiculos: {1: (131, 145.85207096486445), 2: (108, 1304.605971281605), 3: (91, 953.172608610164), 4: (32, 17.302304187458727), 5: (22, 16.627680130757895), 6: (22, 13.602810739289229)}
Oferta: {1: 8.0, 2: 10.0, 3: 0.0, 4: 4.0, 5: 28.0, 6: 3.0, 7: 0.0, 8: 10.0, 9: 43.0, 10: 1.0, 11: 16.0, 12: 18.0}
Demanda: {13: 12.0, 14: 15.0, 15: 15.0, 16: 6.0, 17: 5.0, 18: 11.0, 19: 12.0, 20: 10.0, 21: 15.0}


In [ ]:

# Crear el modelo
model = ConcreteModel()

# Conjuntos e índices
nodos = list(oferta.keys()) + list(demanda.keys())  # El nodo 1 es el depósito
model.N = Set(initialize=nodos)
model.C = Set(initialize=demanda.keys())  # Clientes
model.V = Set(initialize=vehiculos.keys())  # Vehículos
model.D = Set(initialize=oferta.keys())  # Nodo depósito

# Variables
model.x = Var(model.N, model.N, model.V, domain=Binary)  # Ruta (i->j) por vehículo v
model.u = Var(model.N, model.V, domain=NonNegativeReals)  # Carga acumulada en nodo i por vehículo v
model.y = Var(model.V, domain=Binary)  # Indica si el vehículo v es utilizado o no
# Parámetros

# Asumiendo que tus nodos van del 1 al n
model.dist = Param(model.N, model.N, initialize=distancia, default=0)
model.demand = Param(model.C, initialize=demanda)
model.cap = Param(model.V, initialize={v: vehiculos[v][0] for v in vehiculos})
model.range = Param(model.V, initialize={v: vehiculos[v][1] for v in vehiculos})

# Función Objetivo
def obj_rule(m):
    return sum(m.dist[i, j] * m.x[i, j, v] for i in m.N for j in m.N if i != j for v in m.V)
model.obj = Objective(rule=obj_rule, sense=minimize)

# Restricciones

# Cada cliente es visitado exactamente una vez
model.visit_once = ConstraintList()
for c in model.C:
    model.visit_once.add(
        sum(model.x[i, c, v] for i in model.N if i != c for v in model.V) == 1
    )

# Flujo conservado por vehículo en cada cliente
model.flow_conservation = ConstraintList()
for v in model.V:
    for c in model.C:
        model.flow_conservation.add(
            sum(model.x[i, c, v] for i in model.N if i != c) ==
            sum(model.x[c, j, v] for j in model.N if j != c)
        )

# Cada vehículo parte de exactamente un depósito
model.depot_departure = ConstraintList()
for v in model.V:
    model.depot_departure.add(
        sum(model.x[1, j, v] for j in model.N if j != 1) == model.y[v]
    )

# Capacidad del vehículo no debe excederse
model.capacity_constraint = ConstraintList()
for v in model.V:
    model.capacity_constraint.add(
        sum(model.demand[i] * model.x[i, j, v] for i in model.C for j in model.N if i != j) <= model.cap[v]
    )

# Restricción de oferta por depósito
model.supply_limit = ConstraintList()
for d in model.D:
    model.supply_limit.add(
        sum(model.u[d, v] for v in model.V) <= oferta[d]
    )


# Carga inicial de cada vehículo depende del depósito de salida
model.initial_load = ConstraintList()
for v in model.V:
    for d in model.D:
        model.initial_load.add(
            model.u[d, v] <= model.cap[v] * sum(model.x[d, j, v] for j in model.N if j != d)
        )

# La carga nunca puede superar la capacidad del vehículo
model.max_capacity = ConstraintList()
for v in model.V:
    for i in model.N:
        model.max_capacity.add(model.u[i, v] <= model.cap[v])

# Rango del vehículo no debe excederse
model.range_constraint = ConstraintList()
for v in model.V:
    model.range_constraint.add(
        sum(model.dist[i, j] * model.x[i, j, v] for i in model.N for j in model.N if i != j) <= model.range[v]
    )

# No permitir ciclos triviales (i -> i)
model.no_loops = ConstraintList()
for v in model.V:
    for i in model.N:
        model.no_loops.add(model.x[i, i, v] == 0)

# Resolver el modelo
opt = SolverFactory('glpk')
results = opt.solve(model, tee=False)

# Resultados
print("Valor de la función objetivo:", model.obj())
# Mostrar capacidad utilizada por ruta
routes = {}
for v in model.V:
    print(f"Vehículo {v}:")
    routes[v] = []
    for i in model.N:
        for j in model.N:
            if i != j and model.x[i, j, v].value > 0.5:  # Solo si la ruta es tomada
                origen = coord[i]
                destino = coord[j]
                
                if origen not in routes[v]:
                    routes[v].append(origen)
                if destino not in routes[v]:
                    routes[v].append(destino)
                
                if i in model.C:
                    capacidad_utilizada = model.demand[i] * model.x[i, j, v].value
                    print(f"  Ruta {i} -> {j}: Carga transportada = {capacidad_utilizada}")
                else:
                    print(f"  Ruta {i} -> {j} (desde depósito)")



Valor de la función objetivo: 0.0
Vehículo 1:
  Ruta 2 -> 14 (desde depósito)
  Ruta 14 -> 21: Carga transportada = 15.0
  Ruta 17 -> 12: Carga transportada = 5.0
  Ruta 21 -> 17: Carga transportada = 15.0
Vehículo 2:
  Ruta 2 -> 15 (desde depósito)
  Ruta 15 -> 20: Carga transportada = 15.0
  Ruta 20 -> 12: Carga transportada = 10.0
Vehículo 3:
  Ruta 2 -> 13 (desde depósito)
  Ruta 13 -> 16: Carga transportada = 12.0
  Ruta 16 -> 12: Carga transportada = 6.0
Vehículo 4:
  Ruta 2 -> 19 (desde depósito)
  Ruta 19 -> 12: Carga transportada = 12.0
Vehículo 5:
  Ruta 2 -> 18 (desde depósito)
  Ruta 18 -> 12: Carga transportada = 11.0
Vehículo 6:
  Ruta 12 -> 11 (desde depósito)


In [19]:
from pyomo.environ import *

# Crear el modelo
model = ConcreteModel()

# Conjuntos e índices
todos = set(oferta.keys()).union(demanda.keys())
model.N = Set(initialize=todos)
model.D = Set(initialize=oferta.keys())  # Depósitos
model.C = Set(initialize=demanda.keys())  # Clientes
model.V = Set(initialize=vehiculos.keys())  # Vehículos

# Completar distancias faltantes (alta penalización)
for i in todos:
    for j in todos:
        if i != j and (i, j) not in distancia:
            distancia[(i, j)] = 1e6

# Variables
model.x = Var(model.N, model.N, model.V, domain=Binary)
model.u = Var(model.N, model.V, domain=NonNegativeReals)

# Parámetros
model.dist = Param(model.N, model.N, initialize=distancia)
model.demand = Param(model.C, initialize=demanda)
model.cap = Param(model.V, initialize={v: vehiculos[v][0] for v in vehiculos})
model.range = Param(model.V, initialize={v: vehiculos[v][1] for v in vehiculos})

# Función objetivo: minimizar distancia total
def obj_rule(m):
    return sum(m.dist[i, j] * m.x[i, j, v] for i in m.N for j in m.N if i != j for v in m.V)
model.obj = Objective(rule=obj_rule, sense=minimize)

# Cada cliente debe ser visitado una vez
model.visit_once = ConstraintList()
for c in model.C:
    model.visit_once.add(
        sum(model.x[i, c, v] for i in model.N if i != c for v in model.V) == 1
    )

# Conservación de flujo
model.flow_conservation = ConstraintList()
for v in model.V:
    for c in model.C:
        model.flow_conservation.add(
            sum(model.x[i, c, v] for i in model.N if i != c) ==
            sum(model.x[c, j, v] for j in model.N if j != c)
        )

# Cada vehículo parte de exactamente un depósito
model.depot_departure = ConstraintList()
for v in model.V:
    model.depot_departure.add(
        sum(model.x[d, j, v] for d in model.D for j in model.N if j != d) == 1
    )

# Capacidad del vehículo
model.capacity_constraint = ConstraintList()
for v in model.V:
    model.capacity_constraint.add(
        sum(model.demand[i] * model.x[i, j, v] for i in model.C for j in model.N if i != j) <= model.cap[v]
    )

# Carga inicial depende del depósito
model.initial_load = ConstraintList()
for v in model.V:
    for d in model.D:
        model.initial_load.add(
            model.u[d, v] <= model.cap[v] * sum(model.x[d, j, v] for j in model.N if j != d)
        )

# No exceder la capacidad en ningún nodo
model.max_capacity = ConstraintList()
for v in model.V:
    for i in model.N:
        model.max_capacity.add(model.u[i, v] <= model.cap[v])

# Rango del vehículo
model.range_constraint = ConstraintList()
for v in model.V:
    model.range_constraint.add(
        sum(model.dist[i, j] * model.x[i, j, v] for i in model.N for j in model.N if i != j) <= model.range[v]
    )

# No loops (i -> i)
model.no_loops = ConstraintList()
for v in model.V:
    for i in model.N:
        model.no_loops.add(model.x[i, i, v] == 0)

# Flujo de carga acumulada (MTZ modificado)
model.load_flow = ConstraintList()
for v in model.V:
    for i in model.N:
        for j in model.C:
            if i != j:
                model.load_flow.add(
                    model.u[j, v] >= model.u[i, v] + model.demand[j] * model.x[i, j, v] - model.cap[v] * (1 - model.x[i, j, v])
                )

# Resolver el modelo
opt = SolverFactory('glpk')
results = opt.solve(model, tee=False)

# Resultados
print("Valor de la función objetivo:", model.obj())

# Mostrar rutas y capacidad usada
routes = {}
for v in model.V:
    print(f"\nVehículo {v}:")
    routes[v] = []
    for i in model.N:
        for j in model.N:
            if i != j and model.x[i, j, v].value > 0.5:
                origen = coord.get(i)
                destino = coord.get(j)
                if origen and origen not in routes[v]:
                    routes[v].append(origen)
                if destino and destino not in routes[v]:
                    routes[v].append(destino)
                if i in model.C:
                    carga = model.demand[i] * model.x[i, j, v].value
                    print(f"  Ruta {i} -> {j}: Carga = {carga}")
                else:
                    print(f"  Ruta {i} -> {j} (desde depósito)")


(type: set).  This WILL potentially lead to nondeterministic behavior in Pyomo
ERROR: evaluating object as numeric value: x[1,2,1]
        (object: <class 'pyomo.core.base.var.VarData'>)
    No value for uninitialized NumericValue object x[1,2,1]


ValueError: No value for uninitialized NumericValue object x[1,2,1]